In [3]:
int(0b1001001010100101111)

300335

In [14]:
int(0b1011100110100101111)

380207

In [5]:
int(0b1001001010100101111)

300335

In [22]:
[i for i, bits in enumerate(bin(380207)[2:])]

[]

In [328]:
from itertools import starmap
from operator import mul




def v_xormultiplicate_m(v: list, m: list) -> list:
    return [sum(starmap(mul, zip(v, col)))%2 for col in zip(*m)]

In [355]:
# G = [[1,0,1,1,0,0,0],
#       [0,1,0,1,1,0,0], 
#       [0,0,1,0,1,1,0], 
#       [0,0,0,1,0,1,1]] 
#порождающая матрица
G = [
    [0, 1, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 0, 0, 0, 1]
]

H = [
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 1, 1],
    [1, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 0]
]

# H = [[1,0,1],
#       [1,1,1],
#        [1,1,0],
#        [0,1,1],
#        [1,0,0],
#        [0,1,0],
#        [0,0,1]]

#проверочная матрица

In [359]:
#информационное слово (blocksize=4)

d = [1,0,1,1]


In [360]:
#кодовое тогда 
k = v_xormultiplicate_m(d,G)

In [361]:
k

[0, 1, 0, 0, 1, 0, 1, 1]

In [368]:
k = [0, 1, 0, 0, 1, 0, 1, 1]

In [369]:
len(k)

8

In [363]:
# синдром
s = v_xormultiplicate_m(k, H)

In [364]:
s

[1, 1, 0, 0]

In [370]:
si = None
if s != [0, 0, 0]:
    try:
        si = H.index(s)
    except ValueError:
        si = None
if si != None:
    k[si] = int(not k[si])
print(k[4:])

[1, 0, 1, 1]


In [341]:
k[H.index(s)] = int(not k[H.index(s)])

In [342]:
k

[1, 0, 0, 1, 1, 1, 0]

In [285]:

check = 0
for i, row in enumerate(H):
    if sum(s) == 0:
        check = -1
        break
    if s == row:
        check = i

print(check)

0


In [203]:
def list_to_bits(l: list) -> int:
    return int(''.join(map(str, l)), 2)

In [204]:
list_to_bits(s)

4

In [53]:
bin(6)

'0b110'

In [409]:
filein = open('fileIN_encoder.txt', 'rb')
fileout = open('fileOUT_encoder.txt', 'wb')


In [410]:
def encoder(filein,fileout):
    blocksize=4

    G = [
    [0, 1, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 0, 0, 0, 1]
    ]
#порождающая матрица
    while True:
        s=filein.read(blocksize)
        k = [i-48 for i in s]
        coded = v_xormultiplicate_m(k, G)
        #print(coded)
       # print(''.join([chr(i+48) for i in coded]))
        #print(''.join([bin(ord(c))[2:].zfill(8) for c in coded]))
        if not s:
            break
        string_to_send = ''.join(str(c) for c in coded)
        fileout.write(string_to_send.encode())
    fileout.close()

In [411]:
encoder(filein, fileout)
filein.close()
fileout.close()

In [415]:
filein = open('fileOUT_encoder.txt', 'rb')
fileout = open('fileOUT_decoder.txt', 'wb')
filelog = open('fileLOG.log','wb')

In [416]:
#!/usr/bin/python3
from itertools import starmap
from operator import mul


def list_to_bits(l: list) -> int:
    return int(''.join(map(str, l)), 2) << 1

def v_xormultiplicate_m(v: list, m: list) -> list:
    return [sum(starmap(mul, zip(v, col)))%2 for col in zip(*m)]
    
def decoder(filein, fileout, filelog):
    blocksize=8
    s = filein.read(blocksize)
    H = [
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 1, 1],
    [1, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 0]
    ]
    if not s:
        return
    while True:
        filelog.write('\n---ENCODED---\n'.encode())       
        filelog.write(s)
        coded = [int(c)-48 for c in s]
        syndrome = v_xormultiplicate_m(coded, H)
        si = None
        if syndrome != [0,0,0,0]:
            try:
                si = H.index(s)
            except ValueError:
                si = None
        if si != None:
            coded[si] = int(not coded[si])
        log_string = ''.join(str(c) for c in coded)
        string_to_send = ''.join(str(c) for c in coded[4:])
        filelog.write('\n---FULL-DECODED---\n'.encode())
        filelog.write(log_string.encode())
        filelog.write('\n---DECODED---\n'.encode())
        filelog.write(string_to_send.encode())
        fileout.write(string_to_send.encode())
        s = filein.read(blocksize)
        if not s:
            break
    
# si = None
# if s != [0, 0, 0]:
#     try:
#         si = H.index(s)
#     except ValueError:
#         si = None
# if si != None:
#     k[si] = int(not k[si])
# print(k[4:])

In [417]:
decoder(filein, fileout, filelog)
filein.close()
fileout.close()
filelog.close()

In [97]:
bytes('1011')

TypeError: string argument without an encoding

In [129]:
coded = [0, 1, 0, 0, 1, 1, 1]

In [119]:
a = ''.join(str(c) for c in coded)


NameError: name 'ba' is not defined

In [114]:
print('0b'+'0100111')

0b0100111


In [130]:
bitarray(coded)

NameError: name 'bitarray' is not defined

In [133]:
bin(0)

'0b0'

In [121]:
'0100111'.encode()

b'0100111'

In [120]:
bytearray('0100111')

TypeError: string argument without an encoding

In [297]:
Gm = [
    [0, 1, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 0, 0, 0, 1]
]


G = list(zip(*Gm))

In [298]:
Hm = [
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 1, 1],
    [1, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 0]
]

In [299]:
H = list(zip(*Hm))

In [301]:
def mult(a ,b):
    return sum(a[i] * b[i] for i in range(len(a))) % 2

def encode1(data):
    return [mult(data, g) for g in G]

def syndrome(data):
    return [mult(data, H) for h in H]

In [304]:
def decode1(data):
    s = syndrome(data)
    if s != [0, 0, 0, 0]:
        try:
            si = Hm.index(s)
        except ValueError:
            return None

        data[si] = int(not data[si])
    return data[4:]


def i2l(x, z=4):
    return list(map(int, bin(x)[2:].zfill(z)))

def l2i(x):
    return int(''.join(map(str, x)), 2)

def encode_one(x):
    return l2i(encode1(i2l(x,4)))

def decode_one(x):
    return l2i(decode1(i2l(x,8)))

def encode(d):
    ans = []
    for x in d:
        ans.append(encode_one(x >> 4))
        ans.append(encode_one(x & 0b1111))
    return bytes(ans)

def decode(d):
    ans = []
    for i in range(len(d), 2):
        high = decode_one(d[i])
        low = decode_one(d[i+1])
        ans.append(high << 4 | low)
    return bytes(ans)

In [303]:
encode(bytes)

15

In [308]:
type('0100111'.encode())

bytes

In [312]:
encode('0100111'.encode())

b'3\x003\xe13\x003\x003\xe13\xe13\xe1'

In [ ]:
for i in range